## Getting the code of acess

In [12]:
import urllib.parse

CLIENT_ID = "7802i9o56cekpv"               # from Auth tab
REDIRECT  = "http://localhost:8080/callback"
SCOPE     = "r_organization_social"

params = urllib.parse.urlencode(
    {"response_type": "code",
     "client_id": CLIENT_ID,
     "redirect_uri": REDIRECT,
     "scope": SCOPE})

print("Open this URL →", "https://www.linkedin.com/oauth/v2/authorization?" + params)


Open this URL → https://www.linkedin.com/oauth/v2/authorization?response_type=code&client_id=7802i9o56cekpv&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2Fcallback&scope=r_organization_social


## 1 · Exchange the authorization code for a token

This writes a li_token.json containing:

json
Copier
Modifier
{
  "access_token": "AQX…",
  "expires_in": 5183999,
  "expires_at": 174XXX…,
  "refresh_token": "AQX…"
}

In [13]:
import os, requests, json, time

# 1. Paste your code here
CODE = "AQQCmT6PKMV1_sDJigjf_76QnllegbgTOOlfw8kDwKcMGTsg-qyQqyXBN4UF2EsLvb8nRbSP-qeas9MShpY0bBZrWB6QjDaZVsCLKYlTnyoM1Nsa5dWRrWYRRvE4eMOQHn1pg12dv0gHJuAEuIJ2LtYw-xBTZLg8RVwE2AtTIBrhd3Vp0Dtc5jtS7wefeMOBfc9xqJo7fyurOwvYiPM"

CLIENT_ID     = "7802i9o56cekpv"
CLIENT_SECRET = "WPL_AP1.1yum4cW9ZKjzQLSv.zftBJA=="
REDIRECT_URI  = "http://localhost:8080/callback"

resp = requests.post(
    "https://www.linkedin.com/oauth/v2/accessToken",
    data={
        "grant_type":    "authorization_code",
        "code":          CODE,
        "redirect_uri":  REDIRECT_URI,
        "client_id":     CLIENT_ID,
        "client_secret": CLIENT_SECRET,
    }
).json()

# Calculate an absolute expiry timestamp
resp["expires_at"] = time.time() + resp["expires_in"] - 30

# Save to file
with open("li_token.json","w") as f:
    json.dump(resp, f, indent=2)

print("✅ Access token acquired. Expires in", resp["expires_in"], "seconds.")


✅ Access token acquired. Expires in 5183999 seconds.


## 2 · Write a helper to load or refresh the token
Add this function to your script so future runs auto-refresh:

In [7]:
def get_token():
    data = json.load(open("li_token.json"))
    if time.time() < data["expires_at"]:
        return data["access_token"]
    # need to refresh
    refresh_resp = requests.post(
        "https://www.linkedin.com/oauth/v2/accessToken",
        data={
            "grant_type":    "refresh_token",
            "refresh_token": data["refresh_token"],
            "client_id":     CLIENT_ID,
            "client_secret": CLIENT_SECRET,
        }
    ).json()
    # carry forward the same refresh token (LinkedIn returns it only once)
    refresh_resp["refresh_token"] = data["refresh_token"]
    refresh_resp["expires_at"]  = time.time() + refresh_resp["expires_in"] - 30
    json.dump(refresh_resp, open("li_token.json","w"), indent=2)
    return refresh_resp["access_token"]


## 3 · Test the Company Followers endpoint
Now that you have a valid token, run:

In [14]:
# Get company's URN (Uniform Resource Name)
import urllib.parse

token   = get_token()
headers = {"Authorization": f"Bearer {token}"}
vanity  = "airvision"  # part of your page URL: https://www.linkedin.com/company/airvision/
resp    = requests.get(
    "https://api.linkedin.com/v2/organizations"
    f"?q=vanityName&vanityName={urllib.parse.quote(vanity)}",
    headers=headers
).json()

print(resp)  # inspect the JSON
org_id = resp["elements"][0]["id"]
ORG_URN = f"urn:li:organization:{org_id}"
print("Organization URN:", ORG_URN)


{'paging': {'start': 0, 'count': 10, 'links': [], 'total': 1}, 'elements': [{'vanityName': 'airvision', 'localizedName': 'AirVision', 'name': {'localized': {'en_US': 'AirVision'}, 'preferredLocale': {'country': 'US', 'language': 'en'}}, 'primaryOrganizationType': 'NONE', 'locations': [{'locationType': 'HEADQUARTERS', 'address': {'country': 'IL', 'postalCode': '96943'}, 'staffCountRange': 'SIZE_1', 'geoLocation': 'urn:li:geo:101620260'}], 'id': 921718, 'localizedWebsite': 'http://www.airvision.co.il', 'logoV2': {'cropped': 'urn:li:digitalmediaAsset:C560BAQF8owrGvfHh9w', 'original': 'urn:li:digitalmediaAsset:C560BAQF8owrGvfHh9w', 'cropInfo': {'x': 0, 'width': 0, 'y': 0, 'height': 0}}}]}
Organization URN: urn:li:organization:921718


In [10]:
import urllib.parse

ORG_URN = "urn:li:organization:921718"  # replace with your actual org URN
token   = get_token()
headers = {"Authorization": f"Bearer {token}"}

url = (
    "https://api.linkedin.com/v2/organizationalEntityFollowerStatistics"
    "?q=organizationalEntity"
    "&organizationalEntity=" + urllib.parse.quote(ORG_URN)
)

resp = requests.get(url, headers=headers).json()
total_followers = resp["elements"][0]["followerCounts"]["organicFollowerCount"]
print("AirVision page followers:", total_followers)


KeyError: 'elements'

In [9]:
import urllib.parse

ORG_URN = "urn:li:organization:921718"
token   = get_token()
headers = {"Authorization": f"Bearer {token}"}

url = (
    "https://api.linkedin.com/v2/organizationalEntityFollowerStatistics"
    "?q=organizationalEntity"
    "&organizationalEntity=" + urllib.parse.quote(ORG_URN)
)

# Make the request and print out everything
response = requests.get(url, headers=headers)
print("Status code:", response.status_code)
print("Raw response:")
print(response.text)


Status code: 403
Raw response:
{"status":403,"serviceErrorCode":100,"code":"ACCESS_DENIED","message":"Not enough permissions to access: organizationalEntityFollowerStatistics.FINDER-organizationalEntity.NO_VERSION"}


In [11]:
import requests, urllib.parse

ORG_URN = "urn:li:organization:921718"  # make sure this is YOUR page
token   = get_token()                   # your helper that loads the new token

headers = {
    "Authorization": f"Bearer {token}",
    "LinkedIn-Version": "202405",
    "X-Restli-Protocol-Version": "2.0.0",
}

url = (
    "https://api.linkedin.com/rest/organizationalEntityFollowerStatistics"
    "?q=organizationalEntity"
    f"&organizationalEntity={urllib.parse.quote(ORG_URN)}"
)

resp = requests.get(url, headers=headers)
print(resp.status_code, resp.text)      # 200 + JSON when access is approved


426 {"status":426,"code":"NONEXISTENT_VERSION","message":"Requested version 20240501 is not active"}
